In [1]:
import pandas as pd
from scipy import stats

import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from pathlib import Path
import numpy as np
import math

In [2]:

#######################################################################
#        Yingyao Zhou, yzhou@gnf.org
#        August 7, 2013
#        Last modified 8/7/13
#######################################################################

def error_msg(s_msg):
    print("ERROR> ", s_msg)
    exit()

def lnbinomial(n, k):
    return math.lgamma(n+1)-math.lgamma(n-k+1)-math.lgamma(k+1)

def hyper(n, N, n1, n2, tolerance=1e-300, n_chunk=1000):
    '''N M_total: total number of objects in bin
    n1 n_white: total number of white objects in bin
    n2 N_pick: number of draws without replacement
    n x_white: x out of N_pick are white'''
    n_chunk=1000 if n_chunk<=0 else n_chunk
    min_idx=min(n1,n2)
    l_left=n*1.0/n1 < n2*1.0/N and n<min_idx-n+1
    term=1.0
    P=0.0 if l_left else 1.0 #when l_left, do not include pvalue2(N,n1,n2,n) itself
    if l_left:
        for x in range(n-1,-1,-n_chunk):
            ## vectorize in chunks of 1000
            ## in case N is huge, we stop when the remaining area is too small
            if term*(x+1)<tolerance: break # no need to run, too small already
            X=np.arange(x, max(-1, x-n_chunk), -1.0)
            X=(X+1)*(N-n1-n2+1+X)/(n1-X)/(n2-X)
            X=X.cumprod()*term
            term=X[-1]
            P+=X.sum()
    else:
        for x in range(n+1, min_idx+1, n_chunk):
            if term*(min_idx-x+1)<tolerance: break
            X=np.arange(x, min(min_idx+1, x+n_chunk), 1.0)
            X=(1+n1-X)*(1+n2-X)/X/(X+N-n1-n2)
            X=X.cumprod()*term
            term=X[-1]
            P+=X.sum()
    P*=math.exp(lnbinomial(n2,n)+lnbinomial(N-n2,n1-n)-lnbinomial(N,n1))
    return 1.0-P if l_left else P

def hyper_previous(n, N, n1, n2):
    '''N M_total: total number of objects in bin
    n1 n_white: total number of white objects in bin
    n2 N_pick: number of draws without replacement
    n x_white: x out of N_pick are white'''
    min_idx=min(n1,n2)
    l_left=n*1.0/n1 < n2*1.0/N and n<min_idx-n+1
    term=1.0
    P=0.0 if l_left else 1.0 #when l_left, do not include pvalue2(N,n1,n2,n) itself
    if l_left:
        for x in range(n-1,-1,-1):
            term*=(x+1.0)*(N-n2-n1+x+1.0)/(n1-x)/(n2-x)
            P+=term
    else:
        for x in range(n+1, min_idx+1):
            term*=(n1-x+1.0)*(n2-x+1.0)/x/(N-n2-n1+x)
            P+=term
    P*=math.exp(lnbinomial(n2,n)+lnbinomial(N-n2,n1-n)-lnbinomial(N,n1))
    return 1.0-P if l_left else P

def hyper_(x_white, M_total, n_white, N_pick):
    """This version is too slow"""
    '''M_total: total number of objects in bin
    n_white: total number of white objects in bin
    N_pick: number of draws without replacement
    x_white: x out of N_pick are white'''
    return ss.hypergeom.sf(x_white-1, M_total, n_white, N_pick)

def RSA_score(I_rank,N,i_min=None,i_max=None,l_BonferroniCorrection=False):
    cutoff=0
    logP_min=1.0
    n=len(I_rank);
    if i_max is None: i_max=n-1
    if i_min is None: i_min=0
    i_scale=(i_max-i_min+1) if l_BonferroniCorrection else 1
    for i in range(i_min,i_max+1):
        #print i, N, n, I_rank[i]
        #print ">>>>>>>", hyper(i+1, N, n, I_rank[i])*i_scale
        if (i<i_max) and (I_rank[i]==I_rank[i+1]): continue
        logP=math.log(max(hyper(i+1, N, n, I_rank[i]+1)*i_scale, 1e-100), 10)
        if (logP < logP_min):
            logP_min=logP
            cutoff=i
    return {'logP':logP_min, 'cutoff':cutoff}

def RSA(T, s_gene="GeneID", s_score="Score", l_reverse=False, LB=0.2, UB=0.8, l_randomize=False, l_BonferroniCorrection=False):
    t=T.copy()
    S=list(t.columns.values)
    #S=util.header(t)
    if t[s_gene].dtype is not np.dtype(object):
        t[s_gene]=t[s_gene].astype(str)
    t=t[ (pd.notnull(t[s_gene])) & (t[s_gene]!="") & (pd.notnull(t[s_score])) ]
    N=len(t)
    R_logP=np.zeros(N)
    R_bestActivity=np.zeros(N)
    I_hit=np.zeros(N).astype(int)
    I_totWell=np.zeros(N).astype(int)
    I_hitWell=np.zeros(N).astype(int)
    if l_randomize:
        R=t[s_score].values
        R=R[np.random.permutation(len(R))]
        t[s_score]=R
    t.sort_values(by=s_score, ascending=(not l_reverse), inplace=True)
    c_gene=dict()
    c_rank=dict()
    # we need to hash the max rank of a given score.
    # if t is a membership matrix, there are lots of ties, obtaining
    # c_rank can be the bottleneck
    c_score=dict()
    R_score=t[s_score].values
    for i in range(N):
        c_score[R_score[i]]=i

    for i in range(len(t)):
        #s=t.irow(i)[s_gene]
        s=t.iloc[i][s_gene]
        if s not in c_gene:
            c_gene[s]=[] # store the exact index for this gene
            c_rank[s]=[] # modify the rank, in case there are ties
        # updated on 10/19/2012
        c_gene[s].append(i)
        # the following can be the slowest part, if there are lots of ties
        #for j in range(i+1, N):
        #    if t[s_score].iloc[j]!=t[s_score].iloc[i]: break
        #c_rank[s].append(j-1)
        c_rank[s].append(c_score[R_score[i]])
    for s in c_gene:
        #if s!='19218': continue
        I_rank=c_rank[s]
        i_max=None
        i_min=None
        for k in range(len(I_rank)):
            if l_reverse:
                if R_score[I_rank[k]]>=LB: i_max=k
                if R_score[I_rank[k]]>=UB: i_min=k
                if (R_score[I_rank[k]]<LB and i_max is None): i_max=k-1
            else:
                if R_score[I_rank[k]]<=UB: i_max=k
                if R_score[I_rank[k]]<=LB: i_min=k
                if (R_score[I_rank[k]]>UB and i_max is None): i_max=k-1
        rslt=RSA_score(I_rank,N,i_min,i_max,l_BonferroniCorrection=l_BonferroniCorrection)
        logP=rslt['logP']
        cutoff=rslt['cutoff']
        I_idx=c_gene[s]
        for k in range(len(I_idx)):
            R_logP[I_idx[k]]=logP
            R_bestActivity[I_idx[k]]=R_score[I_idx[0]]
            I_hitWell[I_idx[k]]=cutoff+1
            I_totWell[I_idx[k]]=len(I_idx)
            if (k<=cutoff): I_hit[I_idx[k]]=1

    t["LogP"]=R_logP
    t["BestActivity"]=R_bestActivity
    t["RSA_Hit"]=I_hit
    t["#hitWell"]=I_hitWell
    t["#totalWell"]=I_totWell
    # q-value
    t_p=t.drop_duplicates([s_gene,'LogP'])
    Rq=np.log10(adjust_p(np.power(10, t_p.LogP.clip(-np.inf, 0))))
    t_q=pd.DataFrame({s_gene: t_p[s_gene], 'Log_q':Rq})
    t=t.merge(t_q, left_on=[s_gene], right_on=[s_gene])
    ###
    t.sort_values(by=['LogP',s_gene,s_score], ascending=[True, True, not l_reverse], inplace=True)
    #t["LogP"]=util.rarray2sarray(t["LogP"], s_format='%.4f')
    t["RSA_Rank"]=np.zeros(N).astype(int)+999999
    cnt=0
    for k in range(N):
        if t["RSA_Hit"].values[k]>0:
            cnt+=1
            t["RSA_Rank"].values[k]=cnt
    return t

def adjust_p(R_p, N=None, method="BH"):
    """Calculate FDR for multiple test. N is the total # of tests run, if not given, set to len(R_p)
    R_p: an array of p-values
    N: int, total number of tests run
    method: currently fixed to Benjamini and Hochberg method.
    Output has been valided with adjust.p in R"""
    l_old=False # old implementation, slower, keep in case there is bug, new code has been tested
    N=len(R_p) if N is None else N
    if method.upper()=="BONFERRONI":
        return np.clip(np.array(R_p)*N, 0.0, 1.0)
    elif method.upper()=="HOLM":
        n=len(R_p)
        t=pd.DataFrame({'p':R_p, 'q':R_p, 'I':list(range(len(R_p)))})
        t.sort_values('p', ascending=True, inplace=True)
        t.index=range(n)
        if l_old:
            q=0.0
            for i in range(n):
                q=t.ix[i, 'q']=min(max(q, t.ix[i, 'p']*(N-i)),1)
        else:
            q=np.clip(t.p.values*(N-np.arange(n)), 0.0, 1.0)
            q=np.maximum.accumulate(q)
            t['q']=q
        t.sort_values('I', inplace=True)
        return t.q.values
    elif method.upper() in ("BH","FDR"):
        n=len(R_p)
        t=pd.DataFrame({'p':R_p, 'q':R_p, 'I':list(range(len(R_p)))})
        t.sort_values('p', ascending=False, inplace=True)
        t.index=range(n)
        if l_old:
            q=1.0
            for i in range(n):
                q=t.ix[i, 'q']=min(q, t.ix[i, 'p']*N*1.0/(len(t)-i))
        else:
            q=np.clip(t.p.values*N*1.0/(n-np.arange(n)), 0.0, 1.0)
            q=np.minimum.accumulate(q)
            t['q']=q
        t.sort_values('I', inplace=True)
        return t.q.values
    else:
        util.error_msg('Unsupported method: %s' % method)

In [3]:
drug_sensitivity = pd.read_excel("../../Drug_sensitivity/Drug_sensitivity_formatted.xlsx", index_col = "Cell_line")
drug_sensitivity = drug_sensitivity.drop("COSMIC_ID", axis =1)
drug_sensitivity.columns = drug_sensitivity.columns.astype(str)

In [7]:
## Load and zscore metabolites
metabolites_rnaseq_combined = pd.read_csv("../../RNA_correlation/Metabolites_rnaseq_averages_percellline.csv", index_col = "Unnamed: 0")
metabolites_rnaseq_combined = metabolites_rnaseq_combined.drop("MCF7", axis =1)

zscore_dataframe = pd.DataFrame(stats.zscore(metabolites_rnaseq_combined.transpose())).transpose()
zscore_dataframe.columns = metabolites_rnaseq_combined.columns
zscore_dataframe.index = metabolites_rnaseq_combined.index
zscore_dataframe_metabolites = zscore_dataframe.head(1099)

C:\Users\ds728\anaconda3\lib\site-packages\scipy\stats\stats.py:2419: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


In [8]:
zscore_dataframe_metabolites

,SHP77,PC9,LUDLU1,NCIH446,MALME3M,NCIH1755,HCC1187,CORL88,NCIH2171,NCIH1155,...,NCIH526,NCIH146,HL60,NCIH82,SF268,SF539,K562,DU4475,UO31,EKVX
1,0.775782,1.376763,1.177605,1.679806,0.378006,1.916782,2.678167,-0.014645,-0.218838,-1.834040,...,-0.856683,1.072496,-1.482120,0.158258,-0.877183,-1.469532,0.427099,-0.682611,-0.494872,-0.549000
2,1.505271,2.122538,-0.735373,2.862532,-0.971666,-0.869158,0.622574,2.709133,-1.394612,-1.064992,...,0.394029,0.595009,-1.193228,-1.453332,-0.360251,-1.233787,-1.579651,-2.350679,-0.571118,-0.436123
3,0.646809,1.984327,0.547110,1.945590,0.650913,1.761409,3.041286,0.199699,0.112183,2.592212,...,-0.753615,-1.154603,-0.971340,-1.127397,0.760511,1.521197,-0.884878,-1.351474,-1.319922,-0.395378
4,-0.471229,0.819520,-0.186991,0.455024,-0.230821,0.823719,1.008907,-0.918713,-0.225309,1.309784,...,-0.125839,-0.293635,-1.118965,-1.244068,0.609732,1.443811,-0.437810,0.665022,-1.065775,-1.077148
5,0.334760,4.225992,1.277785,0.595728,-0.204668,1.068505,1.273144,1.165229,0.422665,-2.634772,...,0.040395,-0.990317,-0.896629,-0.338983,-0.362405,-0.824628,-1.035282,-1.176537,-0.338405,-1.105548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,0.386287,-0.213819,4.611393,1.572735,2.871780,0.223372,0.406549,-0.265633,-0.820499,0.690981,...,0.134954,1.097238,-1.466104,1.467149,-1.058041,-1.575144,1.436989,0.896687,-1.370726,0.529612
1096,-1.315613,-0.928867,0.011669,-0.279570,-0.090261,-0.241682,-0.223328,-0.942086,0.706502,1.810040,...,0.540113,1.804577,1.575588,2.715725,-0.571837,1.422419,1.016664,0.603479,0.299544,-0.758655
1097,0.339201,-0.026889,0.892945,0.034256,-0.187962,0.434001,-0.358977,0.428238,-0.890822,-0.757267,...,0.134761,-0.249625,-0.658937,-0.815832,-0.464438,-0.853867,-0.642224,-0.767424,-0.797103,-0.139986
1098,-0.725976,-0.146492,1.365924,1.061939,0.897581,0.419725,0.767716,-0.031596,0.662658,2.050792,...,-0.144239,1.699347,0.828493,2.201483,-0.614585,-1.033616,2.046286,1.640022,-0.349499,-1.032364


In [9]:
pathway_annotations = pd.read_csv("../../Metabolic_pathways/Core_Metabolic_pathways_SMPDB.csv")
metabolite_reference = pd.read_csv("../../AZ_data/Metabolite_reference_table.csv")

In [11]:
master_pathwaylist = []
master_correlationlist = []

for pathwaytop in drug_sensitivity.columns:
    print(pathwaytop)
    score_values = pd.DataFrame(drug_sensitivity[pathwaytop]).transpose()
    combined_dataframe = pd.concat([score_values, zscore_dataframe_metabolites], axis =0, join="inner")
    correlation = pd.DataFrame(combined_dataframe.transpose().corr().sort_values(by  =pathwaytop, ascending = False)[pathwaytop])
    correlation = correlation.drop(pathwaytop, axis = 0)
    
    pathway_scores = []
    pathway_correlation = []
    
    ## Calculate the integers for the metabolites in each pathway
    for index, row in pathway_annotations.iterrows():
        pathwayid, pathwayname = row["Pathway ID"], row["Pathway name"]
        pathwayname = pathwayname.replace("/", "_")
        # find the appropriate pathway file
        pathwayfile = ("../../Metabolic_pathways/SMPDB_pathways/" + pathwayid + "_metabolites.csv")
        if Path(pathwayfile).is_file():
            pathway = pd.read_csv(pathwayfile)
            # Remove nans from metabolite ids
            metabolite_ids = pathway["HMDB ID"][~pathway["HMDB ID"].isnull()].tolist()
            ## We have to transform the metabolite ids because they dont match
            new_metabolite_ids = []
            for item in metabolite_ids:
                newid = "HMDB" + item.split("HMDB")[1][2:]
                new_metabolite_ids.append(newid)
            #subset the cell values by this pathway only
            metabolite_idarray = metabolite_reference[metabolite_reference["id"].isin(new_metabolite_ids)]
            metabolite_idlist = metabolite_idarray["ionIdx"].tolist()
        
            correlation["Pathway"] = np.where(correlation.index.astype(int).isin(metabolite_idlist), "path", "nopath")
            if len(correlation["Pathway"].unique()) == 1:
                continue
            ## RSA statistic calculation
            RSAarray =RSA(correlation, s_gene="Pathway", s_score=pathwaytop, l_reverse = True, LB = -1, UB = 1)
            correlation_value = correlation[correlation["Pathway"] == "path"][pathwaytop].mean()
            log_qvalue = RSAarray[RSAarray["Pathway"] == "path"]["Log_q"].iloc[0]
            pathway_scores.append([pathwayname, log_qvalue])
            pathway_correlation.append([pathwayname, correlation_value])
            
    pathway_values = (pd.DataFrame(pathway_scores, columns=["Pathway", "Score_" + pathwaytop]))
    pathway_values = pathway_values.set_index("Pathway")
    master_pathwaylist.append(pathway_values)
    
    correlation_values = (pd.DataFrame(pathway_correlation, columns=["Pathway", "Correlation_" + pathwaytop]))
    correlation_values = correlation_values.set_index("Pathway")
    master_correlationlist.append(correlation_values)

TL-2-105
TAK-715
CP466722
BMS-345541
Genentech Cpd 10
GSK429286A
Ruxolitinib
SB-715992
ZSTK474
KIN001-102
CAL-101
GW-2580
BMS-708163
BX-912
LY317615
WZ3105
XMD14-99
CP724714
JW-7-24-1
ABT-869
GSK2126458
PHA-793887
TG101348
QL-XI-92
XMD15-27
QL-XII-47
AC220
EKB-569
Masitinib
OSI-930
XMD13-2
Bleomycin (50 uM)
AT-7519
XL-184
NPK76-II-72-1
NG-25
TL-1-85
VX-11e
FR-180204
Zibotentan, ZD4054
BIX02189
KIN001-236
KIN001-055
KIN001-266
NVP-BHG712
PIK-93
TPCA-1
Y-39983
YM201636
AV-951
GSK690693
KIN001-270
CH5424802
KIN001-244
KIN001-260
SB52334
AS605240
STF-62247
I-BET
MPS-1-IN-1
CX-5461
Tubastatin A
VNLG/124
XL-880
EX-527
THZ-2-49
PFI-1
UNC0638
UNC0638.1
OSI-027
SN-38
MP470
T0901317
IOX2
5-Fluorouracil
NSC-207895
CAY10603
GSK269962A
SB-505124
Tamoxifen
piperlongumine
PI-103
Phenformin
SNX-2112
BMS-708163.1
BMS-536924
AR-42
EHT 1864
QL-X-138
CCT007093
AG-014699
AZD6244
TW 37
rTRAIL
XAV 939
JNJ-26854165
PLX4720 (rescreen)
CUDC-101
THZ-2-102-1
(5Z)-7-Oxozeaenol
JQ1
CHIR-99021
BMN-673
Afatinib (resc

In [12]:
combined_pathway_dataframe = pd.concat(master_pathwaylist, axis =1)
combined_correlation_dataframe = pd.concat(master_correlationlist, axis =1)

In [13]:
combined_pathway_dataframe.to_csv("Pathway_pvalues_drugsensitivity.csv")
combined_correlation_dataframe.to_csv("Pathway_correlation_drugsensitivity.csv")